# Seldon Core Setup

## Prerequisites

You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [A running Kubernetes cluster with kubectl authenticated](https://kubernetes.io/docs/setup/)
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [Helm client](https://helm.sh/)

## Setup Cluster

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "microk8s" modified.


## Install Seldon Core

Follow the [Seldon Core Install documentation](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html).

In [3]:
!kubectl create namespace seldon-system

namespace/seldon-system created


### If using Ambassador:

In [4]:
!helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set ambassador.enabled=true --set usageMetrics.enabled=true --namespace seldon-system

NAME: seldon-core
LAST DEPLOYED: Sun Feb  6 11:32:49 2022
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 1
TEST SUITE: None


### If using Istio:

In [4]:
!helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set istio.enabled=true --set usageMetrics.enabled=true --namespace seldon-system

NAME: seldon-core
LAST DEPLOYED: Mon Apr 18 10:13:00 2022
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 1
TEST SUITE: None


Check all services are running before proceeding.

### Wait for rollout to finish

In [5]:
!kubectl rollout status deploy/seldon-controller-manager -n seldon-system

deployment "seldon-controller-manager" successfully rolled out


## Install Ingress

### Ambassador

[Ambassador install](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html#install-ambassador)

Ambassador Edge Stack:

In [6]:
!helm repo add datawire https://www.getambassador.io

"datawire" already exists with the same configuration, skipping


In [61]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "kasten" chart repository
...Successfully got an update from the "prometheus-community" chart repository
...Successfully got an update from the "datawire" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈Happy Helming!⎈


In [62]:
!helm install ambassador datawire/ambassador \
    --set image.repository=docker.io/datawire/ambassador \
    --set crds.keep=false \
    --namespace seldon-system

W0128 19:51:19.451053    7699 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0128 19:51:19.609411    7699 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0128 19:51:19.758268    7699 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0128 19:51:19.913691    7699 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0128 19:51:20.063052    7699 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefini

Ambassador API Gateway (Emissary Ingress):

In [6]:
!helm repo add datawire https://www.getambassador.io

"datawire" already exists with the same configuration, skipping


In [7]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "kasten" chart repository
...Successfully got an update from the "datawire" chart repository
...Successfully got an update from the "prometheus-community" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈Happy Helming!⎈


In [8]:
!helm install ambassador datawire/ambassador \
    --set image.repository=docker.io/datawire/ambassador \
    --set crds.keep=false \
    --set enableAES=false \
    --namespace seldon-system

W0206 11:37:23.252209    2271 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0206 11:37:23.395686    2271 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0206 11:37:23.548245    2271 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0206 11:37:23.739815    2271 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0206 11:37:23.978266    2271 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefini

Check all services are running before proceeding.

In [10]:
!kubectl rollout status deployment.apps/ambassador -n seldon-system

Error from server (NotFound): deployments.apps "ambassador" not found


For Ambassador Edge Stack:

**Note:** Absent configuration, Ambassador Edge Stack will attempt to use TLS with a self-signed cert.  Either [set up TLS](https://www.getambassador.io/docs/edge-stack/latest/howtos/tls-termination/) or ignore certificate validation (i.e. `curl -k`).

In [ ]:
!kubectl port-forward $(kubectl get pods -n seldon-system -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon-system 8003:8443

For Ambassador API Gateway:

In [67]:
!kubectl port-forward $(kubectl get pods -n seldon-system -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon-system 8003:8080

Forwarding from 127.0.0.1:8003 -> 8080
Forwarding from [::1]:8003 -> 8080
^C


*or*

### Istio

[Istio install](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html#ingress-support)

**Note:** Remember to add ```--set istio.enabled=true``` flag when installing Seldon Core with Istio Ingress.